# Introduction to Geospatial Data Science - Final Assignment (webscraping script)

**Author: Márton Nagy**

**Note: This is only the webscraping script. For the actual analysis, please refer to the other uploaded notebook.**

In [1]:
import pandas as pd
import numpy as np
import json
import requests
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime

For this assignment, I scrape data from Willhaben, Austria's largest online marketplace. I scrape all available data on properties listed for sale on the site in Vienna. The data was scraped on 17 March 2025.

To perform the scraping, I first define a function that fetches one page from the Willhaben API.

In [2]:
def get_page(page):
    headers = {
        'accept': 'application/json',
        'accept-language': 'en-US,en;q=0.9',
        'dnt': '1',
        'priority': 'u=1, i',
        'referer': 'https://www.willhaben.at/iad/immobilien/eigentumswohnung/wien',
        'sec-ch-ua': '"Chromium";v="134", "Not:A-Brand";v="24", "Google Chrome";v="134"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-model': '""',
        'sec-ch-ua-platform': '"Windows"',
        'sec-ch-ua-platform-version': '"19.0.0"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36',
        'x-bbx-csrf-token': '5c361365-5e65-4ddd-ac5b-05a30a43771c',
        'x-wh-client': 'api@willhaben.at;responsive_web;server;1.0.0;desktop',
    }

    params = {
        'sfId': 'd6dc04bc-1d8f-418d-9abb-3ffb42cd429e',
        'page': str(page),
        'rows' : '150'
    }

    response = requests.get(
        'https://www.willhaben.at/webapi/iad/search/atz/seo/immobilien/eigentumswohnung/wien',
        params=params,
        headers=headers,
    )

    return json.loads(response.text)['advertSummaryList']['advertSummary']

As the API can only return 150 results per page at maximum, I iterate through all the pages until I get an error.

In [4]:
i = 1
json_list = []
while True:
    try:
        json_list.extend(get_page(i))
        i += 1
    except:
        break

I save the results to a JSON file for later use.

In [5]:
json.dump(json_list, open('willhaben.json', 'w', encoding='utf-8'), ensure_ascii=False)

I read the saved JSON file.

In [19]:
json_estate_data = json.load(open('willhaben.json', 'r', encoding='utf-8'))

I scraped 16,981 listings which matches the number of properties indicated on the website, meaning that the scraping was successful.

In [20]:
len(json_estate_data)

16981

The data is in a well-structured JSON format, so I need to extract relevant fields into a DataFrame.

In [21]:
def extract_attribute(attributes, key, return_list=False):
    """Helper function to extract attribute values based on the key."""
    for attr in attributes.get('attribute', []):
        if attr['name'] == key:
            if return_list:
                return attr['values'] if attr['values'] else []
            return attr['values'][0] if attr['values'] else None
    return [] if return_list else None
        
data = []

for estate in json_estate_data:
    attributes = estate.get('attributes', {})
    advert_status = estate.get('advertStatus', {}).get('id', '')
    coordinates = extract_attribute(attributes, 'COORDINATES')

    # Split coordinates into latitude and longitude
    latitude, longitude = (coordinates.split(',') if coordinates else (np.nan, np.nan))

    data.append({
        'id': estate.get('id', ''),
        'url': 'https://www.willhaben.at/iad/' + extract_attribute(attributes, 'SEO_URL'),
        'price': extract_attribute(attributes, 'PRICE'),
        'is_active': 1 if advert_status == 'active' else 0,
        'description': extract_attribute(attributes, 'BODY_DYN'),
        'location': extract_attribute(attributes, 'LOCATION'),
        'country': extract_attribute(attributes, 'COUNTRY'),
        'state': extract_attribute(attributes, 'STATE'),
        'district': extract_attribute(attributes, 'DISTRICT'),
        'address' : extract_attribute(attributes, 'ADDRESS'),
        'postcode': extract_attribute(attributes, 'POSTCODE'),
        'latitude': float(latitude) if latitude else None,
        'longitude': float(longitude) if longitude else None,
        'advertiser' : extract_attribute(attributes, 'ORGNAME'),
        'advertiser_id' : extract_attribute(attributes, 'ORGID'),
        'property_type' : extract_attribute(attributes, 'PROPERTY_TYPE'),
        'is_private' : extract_attribute(attributes, 'ISPRIVATE'),
        'is_flat' : extract_attribute(attributes, 'PROPERTY_TYPE_FLAT'),
        'location_quality' : extract_attribute(attributes, 'LOCATION_QUALITY'),
        'floor' : extract_attribute(attributes, 'FLOOR'),
        'rooms' : extract_attribute(attributes, 'NUMBER_OF_ROOMS'),
        'size' : extract_attribute(attributes, 'ESTATE_SIZE'),
        'published_date' : extract_attribute(attributes, 'PUBLISHED_String'),
        'area_by_type_dict' : dict(zip(extract_attribute(attributes, 'FREE_AREA_TYPE_NAME', return_list=True),
                                       extract_attribute(attributes, 'FREE_AREA_TYPE', return_list=True)[0].split(', ')
                                       if len(extract_attribute(attributes, 'FREE_AREA_TYPE', return_list=True)) > 0 else []))
    })

df = pd.DataFrame(data)

I manage type conversions and missing values here.

In [22]:
df['price'] = df.price.apply(lambda x: np.nan if pd.isna(x) else float(x) if x.isnumeric() else np.nan)
df['address'] = df.address.apply(lambda x: np.nan if pd.isna(x) else x)
df['postcode'] = df.postcode.apply(lambda x: np.nan if pd.isna(x) else x)
df['advertiser'] = df.advertiser.apply(lambda x: np.nan if pd.isna(x) else x)
df['property_type'] = df.property_type.apply(lambda x: np.nan if pd.isna(x) else x)
df['is_private'] = df.is_private.apply(lambda x: np.nan if pd.isna(x) else (1 if x == '1' else 0))
df['is_flat'] = df.is_flat.apply(lambda x: np.nan if pd.isna(x) else (1 if x == 'true' else 0))
df['location_quality'] = df.location_quality.apply(lambda x: np.nan if pd.isna(x) else float(x))
df['floor'] = df.floor.apply(lambda x: np.nan if pd.isna(x) else float(x) if x.isnumeric() else np.nan)
df['rooms'] = df.rooms.apply(lambda x: np.nan if pd.isna(x) else float(x) if x.isnumeric() else np.nan)
df['size'] = df['size'].apply(lambda x: np.nan if pd.isna(x) else float(x) if x.isnumeric() else np.nan)
df['published_date'] = pd.to_datetime(df.published_date, errors='coerce')

The data contains information about certain property extensions, like gardens or balconies and their sizes in $m^{2}$. I expand these into separate columns.

In [23]:
df_expanded = df['area_by_type_dict'].apply(pd.Series).fillna(0).astype(int)
df = pd.concat([df.drop(columns=['area_by_type_dict']), df_expanded], axis=1)

In [24]:
df.rename(columns={
    'Terrasse': 'terrace_size',
    'Balkon': 'balcony_size',
    'Loggia': 'loggia_size',
    'Garten': 'garden_size',
    'Dachterrasse': 'roof_terrace_size',
    'Wintergarten': 'winter_garden_size',
}, inplace=True)

There were some duplicate values in the dataset, so I drop these.

In [25]:
df.drop_duplicates(subset = ['id'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [26]:
df.describe(percentiles=[0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]).T.round(2)

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
price,16065.0,584540.21,743957.46,1.00,115000.00,185000.00,221142.20,290000.00,399000.00,599000.00,999900.00,1500000.00,3337000.00,15900000.00
is_active,16839.0,1.00,0.03,0.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
latitude,16595.0,48.21,0.03,48.12,48.14,48.15,48.17,48.19,48.21,48.24,48.25,48.26,48.29,48.30
longitude,16595.0,16.37,0.06,16.20,16.22,16.27,16.30,16.33,16.37,16.41,16.45,16.49,16.52,16.54
is_private,16839.0,0.02,0.13,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00
is_flat,16242.0,1.00,0.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
location_quality,16595.0,0.95,0.17,0.00,0.00,0.69,0.90,1.00,1.00,1.00,1.00,1.00,1.00,1.00
floor,12379.0,2.95,4.25,0.00,0.00,0.00,0.00,1.00,2.00,4.00,5.00,7.00,29.22,45.00
rooms,16839.0,2.51,1.25,0.00,0.00,0.00,1.00,2.00,2.00,3.00,4.00,4.00,6.00,45.00
size,16176.0,77.07,51.46,13.00,30.00,35.00,40.00,49.00,65.00,88.00,122.00,155.00,266.00,1566.00


I calculate the age of the listing and the price per $m^{2}$. I drop the dummy for flats, as it has no variation.

In [27]:
df['published_date'] = df['published_date'].dt.tz_localize(None)
df['ad_age'] = (datetime.now() - df['published_date']).dt.days
df['m2_price'] = df['price'] / df['size']
df.drop(columns=['is_flat'], inplace=True)

As the API does not contain any information on the age of the properties, I create dummies based on the description for newly built listings. I look for *Erstbezug* meaning first move-in. First move-in sometimes means that it is the first move-in after e.g. renovation, so I filter these out as well. I also create a dummy for renovations by looking for *renoviert* (meaning renovated) and *Erstbezug nach*, meaning first move-in after something.

In [28]:
df['is_new_build'] = df['description'].apply(lambda x: 1 if (('erstbezug' in x.lower() and 'erstbezug nach' not in x.lower())) else 0)
df['is_renovated'] = df['description'].apply(lambda x: 1 if ('renoviert' in x.lower() or 'erstbezug nach' in x.lower()) else 0)

In [29]:
df.describe(percentiles=[0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]).T.round(2)

,count,mean,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max,std
price,16065.0,584540.206598,1.0,115000.0,185000.0,221142.2,290000.0,399000.0,599000.0,999900.0,1500000.0,3337000.0,15900000.0,743957.460869
is_active,16839.0,0.999109,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.029834
latitude,16595.0,48.211114,48.124403,48.136087,48.15163,48.170341,48.188838,48.21021,48.238647,48.253769,48.264067,48.287078,48.302486,0.034012
longitude,16595.0,16.370151,16.198876,16.216754,16.274781,16.297091,16.328711,16.369266,16.405251,16.447838,16.488994,16.52014,16.539982,0.060832
is_private,16839.0,0.017638,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.131634
location_quality,16595.0,0.953033,0.0,0.0,0.69,0.9,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.167677
floor,12379.0,2.947653,0.0,0.0,0.0,0.0,1.0,2.0,4.0,5.0,7.0,29.22,45.0,4.254469
rooms,16839.0,2.50585,0.0,0.0,0.0,1.0,2.0,2.0,3.0,4.0,4.0,6.0,45.0,1.254264
size,16176.0,77.072948,13.0,30.0,35.0,40.0,49.0,65.0,88.0,122.0,155.0,266.0,1566.0,51.464088
published_date,16839,2025-03-12 01:54:25.025298432,2024-05-20 00:00:00,2025-02-19 03:22:00,2025-03-03 18:59:48,2025-03-05 17:31:00,2025-03-11 10:37:00,2025-03-12 20:39:00,2025-03-14 15:07:00,2025-03-17 03:29:00,2025-03-17 12:16:00,2025-03-17 15:33:00,2025-03-17 16:27:00,NaN


Lastly, I pickle the compiled dataset for later use. Note that cleaning and filtering the dataset will be handled in the analysis script.

In [30]:
pd.to_pickle(df, 'willhaben.pkl')